# Create pf3k release5

This notebook was run from malsrv2

This notebook requires the output from notebook 20160120_vqsr_delta_exploration.ipynb (which was run on Richard'd MacBook, and results copied over to malsrv2)

In [1]:
%run imports.ipynb
%run _shared_setup.ipynb

In [36]:
import stat
from sh import ssh
farm3 = ssh.bake("farm3-login")
bsub = sh.Command('bsub')

In [2]:
# !apt-get update
# !apt-get install tabix

# Setup

In [18]:
release5_intermediate_files_dir = '/nfs/team112_internal/rp7/data/pf3k/release5/intermediate_files'
release5_intermediate_files_dir = '/lustre/scratch110/malaria/rp7/data/pf3k/release5/intermediate_files'
release5_final_files_dir = '/nfs/team112_internal/production/release_build/Pf3K/pilot_5_0'
output_dir = '/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm'
if not os.path.exists(output_dir + '/scripts'):
    os.makedirs(output_dir + '/scripts')

In [4]:
if not os.path.exists(release5_intermediate_files_dir):
    os.makedirs(release5_intermediate_files_dir)
if not os.path.exists(os.path.dirname(CHROM_VCF_FNS['release5']['Pf3D7_01_v3'])):
    os.makedirs(os.path.dirname(CHROM_VCF_FNS['release5']['Pf3D7_01_v3']))

In [5]:
CHROM_VCF_FNS['release5']['Pf3D7_01_v3']

'/nfs/team112_internal/production/release_build/Pf3K/pilot_5_0/SNP_INDEL_Pf3D7_01_v3.combined.filtered.vcf.gz'

# Plan
- Make SNP and indel files for each chrom using SelectVariants
- Add VQSLOD scores to these using ApplyRecalibration with outputs of 20160120_vqsr_delta_exploration.ipynb
- Combine SNP and indel files using CombineVariants
- Create final filtered versions using VariantFiltration
- Merge all chroms using bcftools concat

In [38]:
GenomeAnalysisTK="/software/jre1.7.0_25/bin/java -Xmx4G -jar /nfs/team112_internal/production/tools/bin/gatk/GenomeAnalysisTK-3.5/GenomeAnalysisTK.jar"
bcftools="/nfs/team112_internal/production/tools/bin/bcftools_git"

In [7]:
!{GenomeAnalysisTK}

##### ERROR ------------------------------------------------------------------------------------------
##### ERROR A USER ERROR has occurred (version 3.5-0-g36282e4): 
##### ERROR
##### ERROR This means that one or more arguments or inputs in your command are incorrect.
##### ERROR The error message below tells you what is the problem.
##### ERROR
##### ERROR If the problem is an invalid argument, please check the online documentation guide
##### ERROR (or rerun your command with --help) to view allowable command-line arguments for this tool.
##### ERROR
##### ERROR Visit our website and forum for extensive documentation and answers to 
##### ERROR commonly asked questions http://www.broadinstitute.org/gatk
##### ERROR
##### ERROR Please do NOT post this error to the GATK forum unless you have really tried to fix it yourself.
##### ERROR
##### ERROR MESSAGE: Argument with name '--analysis_type' (-T) is missing.
##### ERROR ---------------------------------------------------------------

In [8]:
!{bcftools}


Program: bcftools (Tools for variant calling and manipulating VCFs and BCFs)
Version: 1.1-113-gd991f3f (using htslib 1.1-104-g948a68c)

Usage:   bcftools <command> <argument>

Commands:

 -- Indexing
    index        index VCF/BCF files

 -- VCF/BCF manipulation
    annotate     annotate and edit VCF/BCF files
    concat       concatenate VCF/BCF files from the same set of samples
    convert      convert VCF/BCF files to different formats and back
    isec         intersections of VCF/BCF files
    merge        merge VCF/BCF files files from non-overlapping sample sets
    norm         left-align and normalize indels
    plugin       user-defined plugins
    query        transform VCF/BCF into user-defined formats
    reheader     modify VCF/BCF header, change sample names
    view         VCF/BCF conversion, view, subset and filter VCF/BCF files

 -- VCF/BCF analysis
    call         SNP/indel calling
    consensus    create consensus sequence by applying VCF variants
    filter    

In [9]:
chroms = CHROM_VCF_FNS['release5'].keys()

In [14]:
list(chroms)[0:10]

['Pf3D7_01_v3',
 'Pf3D7_02_v3',
 'Pf3D7_03_v3',
 'Pf3D7_04_v3',
 'Pf3D7_05_v3',
 'Pf3D7_06_v3',
 'Pf3D7_07_v3',
 'Pf3D7_08_v3',
 'Pf3D7_09_v3',
 'Pf3D7_10_v3']

In [ ]:
# Make SNP and indel files for each chrom using SelectVariants

for chrom in chroms:
    input_vcf_fn = CHROM_VCF_FNS['interim5'][chrom]
    snps_fn = "%s/snps_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    indels_fn = "%s/indels_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    if not os.path.exists("%s.tbi" % input_vcf_fn):
        !bcftools index --tbi {input_vcf_fn}
    !{GenomeAnalysisTK} \
        -T SelectVariants \
        -R {GENOME_FN} \
        -V {input_vcf_fn} \
        -selectType SNP \
        -o /dev/stdout | bgzip > {snps_fn}
    !{bcftools} index --tbi {snps_fn}
    !{GenomeAnalysisTK} \
        -T SelectVariants \
        -R {GENOME_FN} \
        -V {input_vcf_fn} \
        -xlSelectType SNP \
        -o /dev/stdout | bgzip > {indels_fn}
    !{bcftools} index --tbi {indels_fn}


INFO  12:12:01,662 HelpFormatter - -------------------------------------------------------------------------------- 
INFO  12:12:01,668 HelpFormatter - The Genome Analysis Toolkit (GATK) v3.5-0-g36282e4, Compiled 2015/11/25 04:03:56 
INFO  12:12:01,669 HelpFormatter - Copyright (c) 2010 The Broad Institute 
INFO  12:12:01,669 HelpFormatter - For support and documentation go to http://www.broadinstitute.org/gatk 
INFO  12:12:01,674 HelpFormatter - Program Args: -T SelectVariants -R /lustre/scratch109/malaria/pf3k_methods/resources/Pfalciparum.genome.fasta -V /lustre/scratch109/malaria/pf3k_methods/output/1/f/5/c/394734/1_gatk_variant_filter_gatk3/SNP_INDEL_Pf3D7_01_v3.combined.filtered.vcf.gz -selectType SNP -o /dev/stdout 
INFO  12:12:01,682 HelpFormatter - Executing as rp7@malsrv2 on Linux 3.2.0-75-generic amd64; Java HotSpot(TM) 64-Bit Server VM 1.7.0_25-b15. 
INFO  12:12:01,683 HelpFormatter - Date/Time: 2016/02/04 12:12:01 
INFO  12:12:01,683 HelpFormatter - -----------------------

In [40]:
tranches_snp_fn = '/lustre/scratch110/malaria/rp7/data/pf3k/interim5/vqsr/no_ranksum_3_5_mqcap70/SNP.tranches'
recal_snp_fn = '/lustre/scratch110/malaria/rp7/data/pf3k/interim5/vqsr/no_ranksum_3_5_mqcap70/SNP.recal'
tranches_indel_fn = '/lustre/scratch110/malaria/rp7/data/pf3k/interim5/vqsr/gatk_rec_indel/INDEL.tranches'
recal_indel_fn = '/lustre/scratch110/malaria/rp7/data/pf3k/interim5/vqsr/gatk_rec_indel/INDEL.recal'

for chrom in chroms:
    snps_fn = "%s/snps_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    indels_fn = "%s/indels_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    snps_recalibrated_fn = "%s/snps_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    indels_recalibrated_fn = "%s/indels_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    
    if not os.path.exists(snps_recalibrated_fn):
        script_fn = "%s/scripts/ApplyRecalibration_SNP_%s.sh" % (output_dir, chrom)
        fo = open(script_fn, 'w')
        print(script_fn)
        print('''\
%s \
-T ApplyRecalibration \
-R %s \
-input %s \
-tranchesFile %s \
-recalFile %s \
--ts_filter_level 99.5 \
-mode SNP \
--ignore_all_filters \
-o /dev/stdout | bgzip > %s
''' % (
                GenomeAnalysisTK,
                GENOME_FN,
                snps_fn,
                tranches_snp_fn,
                recal_snp_fn,
                snps_recalibrated_fn
            ),
            file = fo
        )
        fo.close()
        st = os.stat(script_fn)
        os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
#         farm3.bsub(
        bsub(
            '-G', 'malaria-dk',
            '-P', 'malaria-dk',
            '-q', 'normal',
            '-o', '%s/%s_SNP.out' % (output_dir, chrom),
            '-e', '%s/%s_SNP.err' % (output_dir, chrom),
            '-J', '%s_SNP' % (chrom),
            '-R', "'select[mem>8000] rusage[mem=8000]'",
            '-M', '8000',
            script_fn)

    if not os.path.exists(indels_recalibrated_fn):
        script_fn = "%s/scripts/ApplyRecalibration_INDEL_%s.sh" % (output_dir, chrom)
        fo = open(script_fn, 'w')
        print(script_fn)
        print('''\
%s \
-T ApplyRecalibration \
-R %s \
-input %s \
-tranchesFile %s \
-recalFile %s \
--ts_filter_level 99.0 \
-mode INDEL \
--ignore_all_filters \
-o /dev/stdout | bgzip > %s
''' % (
                GenomeAnalysisTK,
                GENOME_FN,
                indels_fn,
                tranches_indel_fn,
                recal_indel_fn,
                indels_recalibrated_fn
            ),
            file = fo
        )
        fo.close()
        st = os.stat(script_fn)
        os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
#         farm3.bsub(
        bsub(
            '-G', 'malaria-dk',
            '-P', 'malaria-dk',
            '-q', 'normal',
            '-o', '%s/%s_INDEL.out' % (output_dir, chrom),
            '-e', '%s/%s_INDEL.err' % (output_dir, chrom),
            '-J', '%s_IND' % (chrom),
            '-R', "'select[mem>8000] rusage[mem=8000]'",
            '-M', '8000',
            script_fn)


/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/ApplyRecalibration_INDEL_Pf3D7_08_v3.sh


In [41]:
for chrom in chroms:
    snps_recalibrated_fn = "%s/snps_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    indels_recalibrated_fn = "%s/indels_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    
    if not os.path.exists("%s.tbi" % snps_recalibrated_fn):
        script_fn = "%s/scripts/index_ApplyRecalibration_SNP_%s.sh" % (output_dir, chrom)
        fo = open(script_fn, 'w')
        print(script_fn)
        print('%s index --tbi %s' % (bcftools, snps_recalibrated_fn), file = fo)
        fo.close()
        st = os.stat(script_fn)
        os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
        bsub(
            '-G', 'malaria-dk',
            '-P', 'malaria-dk',
            '-q', 'normal',
            '-o', '%s/iAR_%s_SNP.out' % (output_dir, chrom),
            '-e', '%s/iAR_%s_SNP.err' % (output_dir, chrom),
            '-J', 'iAR_%s_SNP' % (chrom),
            '-R', "'select[mem>4000] rusage[mem=4000]'",
            '-M', '4000',
            script_fn)

    if not os.path.exists("%s.tbi" % indels_recalibrated_fn):
        script_fn = "%s/scripts/index_ApplyRecalibration_INDEL_%s.sh" % (output_dir, chrom)
        fo = open(script_fn, 'w')
        print(script_fn)
        print('%s index --tbi %s' % (bcftools, indels_recalibrated_fn), file = fo)
        fo.close()
        st = os.stat(script_fn)
        os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
        bsub(
            '-G', 'malaria-dk',
            '-P', 'malaria-dk',
            '-q', 'normal',
            '-o', '%s/iAR_%s_IND.out' % (output_dir, chrom),
            '-e', '%s/iAR_%s_IND.err' % (output_dir, chrom),
            '-J', 'iAR_%s_IND' % (chrom),
            '-R', "'select[mem>4000] rusage[mem=4000]'",
            '-M', '4000',
            script_fn)



/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/index_ApplyRecalibration_SNP_Pf3D7_01_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/index_ApplyRecalibration_INDEL_Pf3D7_01_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/index_ApplyRecalibration_SNP_Pf3D7_02_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/index_ApplyRecalibration_INDEL_Pf3D7_02_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/index_ApplyRecalibration_SNP_Pf3D7_03_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/index_ApplyRecalibration_INDEL_Pf3D7_03_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/index_ApplyRecalibration_SNP_Pf3D7_04_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/index_ApplyRec

In [46]:
for chrom in chroms:
    snps_recalibrated_fn = "%s/snps_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    indels_recalibrated_fn = "%s/indels_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    combined_recalibrated_fn = "%s/snps_indels_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    
    if not os.path.exists(combined_recalibrated_fn):
        script_fn = "%s/scripts/CombineVariants_%s.sh" % (output_dir, chrom)
        fo = open(script_fn, 'w')
        print(script_fn)
        print('''\
%s \
-T CombineVariants \
-R %s \
--variant:snp %s \
--variant:indel %s \
-genotypeMergeOptions PRIORITIZE \
-priority snp,indel \
-o /dev/stdout | bgzip > %s

%s index --tbi %s
''' % (
                GenomeAnalysisTK,
                GENOME_FN,
                snps_recalibrated_fn,
                indels_recalibrated_fn,
                combined_recalibrated_fn,
                bcftools,
                combined_recalibrated_fn
            ),
            file = fo
        )
        fo.close()
        st = os.stat(script_fn)
        os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
        bsub(
            '-G', 'malaria-dk',
            '-P', 'malaria-dk',
            '-q', 'normal',
            '-o', '%s/CV_%s_SNP.out' % (output_dir, chrom),
            '-e', '%s/CV_%s_SNP.err' % (output_dir, chrom),
            '-J', 'CV_%s_SNP' % (chrom),
            '-R', "'select[mem>8000] rusage[mem=8000]'",
            '-M', '8000',
            script_fn)


/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_01_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_02_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_03_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_04_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_05_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_06_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_07_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_08_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scri

In [46]:
for chrom in chroms:
    snps_recalibrated_fn = "%s/snps_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    indels_recalibrated_fn = "%s/indels_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    combined_recalibrated_fn = "%s/snps_indels_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    
    if not os.path.exists(combined_recalibrated_fn):
        script_fn = "%s/scripts/CombineVariants_%s.sh" % (output_dir, chrom)
        fo = open(script_fn, 'w')
        print(script_fn)
        print('''\
%s \
-T CombineVariants \
-R %s \
--variant:snp %s \
--variant:indel %s \
-genotypeMergeOptions PRIORITIZE \
-priority snp,indel \
-o /dev/stdout | bgzip > %s

%s index --tbi %s
''' % (
                GenomeAnalysisTK,
                GENOME_FN,
                snps_recalibrated_fn,
                indels_recalibrated_fn,
                combined_recalibrated_fn,
                bcftools,
                combined_recalibrated_fn
            ),
            file = fo
        )
        fo.close()
        st = os.stat(script_fn)
        os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
        bsub(
            '-G', 'malaria-dk',
            '-P', 'malaria-dk',
            '-q', 'normal',
            '-o', '%s/CV_%s_SNP.out' % (output_dir, chrom),
            '-e', '%s/CV_%s_SNP.err' % (output_dir, chrom),
            '-J', 'CV_%s_SNP' % (chrom),
            '-R', "'select[mem>8000] rusage[mem=8000]'",
            '-M', '8000',
            script_fn)


/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_01_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_02_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_03_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_04_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_05_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_06_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_07_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/CombineVariants_Pf3D7_08_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scri

In [52]:
# Note that unlike previous run, this time we are fixing AD Number=R in the header
# for chrom in ['Pf_M76611', 'Pf3D7_API_v3']:
# for chrom in ['Pf3D7_%02d_v3' % n for n in range(1,6)]:
for chrom in chroms:
    combined_recalibrated_fn = "%s/snps_indels_recalibrated_%s.vcf.gz" % (release5_intermediate_files_dir, chrom)
    combined_filtered_fn = "%s/SNP_INDEL_%s.combined.filtered.vcf.gz" % (release5_final_files_dir, chrom)

    if not os.path.exists(combined_filtered_fn):
        script_fn = "%s/scripts/VariantFiltration_%s.sh" % (output_dir, chrom)
        fo = open(script_fn, 'w')
        print(script_fn)
        print('''\
%s \
-T VariantFiltration \
-R %s \
--variant %s \
--filterName "Low_VQSLOD" --filterExpression "VQSLOD<=0.0" \
--filterName "Centromere" --filterExpression "RegionType=='Centromere'" \
--filterName "InternalHypervariable" --filterExpression "RegionType=='InternalHypervariable'" \
--filterName "SubtelomericHypervariable" --filterExpression "RegionType=='SubtelomericHypervariable'" \
--filterName "SubtelomericRepeat" --filterExpression "RegionType=='SubtelomericRepeat'" \
--invalidatePreviousFilters \
-o /dev/stdout | sed 's/##FORMAT=<ID=AD,Number=./##FORMAT=<ID=AD,Number=R/' | bgzip > %s

%s index --tbi %s
''' % (
                GenomeAnalysisTK,
                GENOME_FN,
                combined_recalibrated_fn,
                combined_filtered_fn,
                bcftools,
                combined_filtered_fn
            ),
            file = fo
        )
        fo.close()
        st = os.stat(script_fn)
        os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
        bsub(
            '-G', 'malaria-dk',
            '-P', 'malaria-dk',
            '-q', 'normal',
            '-o', '%s/VF_%s_SNP.out' % (output_dir, chrom),
            '-e', '%s/VF_%s_SNP.err' % (output_dir, chrom),
            '-J', 'VF_%s_SNP' % (chrom),
            '-R', "'select[mem>8000] rusage[mem=8000]'",
            '-M', '8000',
            script_fn)

#         !GenomeAnalysisTK \
#             -T  \
#             -R {GENOME_FN} \
#             --variant {combined_recalibrated_fn} \
#             --filterName "Low_VQSLOD" --filterExpression "VQSLOD<=0.0" \
#             --filterName "Centromere" --filterExpression "RegionType=='Centromere'" \
#             --filterName "InternalHypervariable" --filterExpression "RegionType=='InternalHypervariable'" \
#             --filterName "SubtelomericHypervariable" --filterExpression "RegionType=='SubtelomericHypervariable'" \
#             --filterName "SubtelomericRepeat" --filterExpression "RegionType=='SubtelomericRepeat'" \
#             --invalidatePreviousFilters \
#             -o /dev/stdout | bgzip > {combined_filtered_fn}

#     if not os.path.exists("%s.tbi" % combined_filtered_fn):
#         !bcftools index --tbi {combined_filtered_fn}


/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/VariantFiltration_Pf3D7_06_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/VariantFiltration_Pf3D7_07_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/VariantFiltration_Pf3D7_08_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/VariantFiltration_Pf3D7_09_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/VariantFiltration_Pf3D7_10_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/VariantFiltration_Pf3D7_11_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/VariantFiltration_Pf3D7_12_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_release_5_farm/scripts/VariantFiltration_Pf3D7_13_v3.sh
/nfs/team112_internal/rp7/data/pf3k/analysis/20160204_create_rel

In [54]:
# Copy these to external ftp site
!rsync -av /nfs/team112_internal/production/release_build/Pf3K/pilot_5_0/SNP* \
web-bfint:/data/production/pf3k/release_5/5.1/

LD_LIBRARY_PATH: Undefined variable.
sending incremental file list
SNP_INDEL_Pf3D7_01_v3.combined.filtered.vcf.gz
SNP_INDEL_Pf3D7_01_v3.combined.filtered.vcf.gz.tbi
SNP_INDEL_Pf3D7_02_v3.combined.filtered.vcf.gz
SNP_INDEL_Pf3D7_02_v3.combined.filtered.vcf.gz.tbi
SNP_INDEL_Pf3D7_03_v3.combined.filtered.vcf.gz
SNP_INDEL_Pf3D7_03_v3.combined.filtered.vcf.gz.tbi
SNP_INDEL_Pf3D7_04_v3.combined.filtered.vcf.gz
SNP_INDEL_Pf3D7_04_v3.combined.filtered.vcf.gz.tbi
SNP_INDEL_Pf3D7_05_v3.combined.filtered.vcf.gz
SNP_INDEL_Pf3D7_05_v3.combined.filtered.vcf.gz.tbi
SNP_INDEL_Pf3D7_06_v3.combined.filtered.vcf.gz
SNP_INDEL_Pf3D7_06_v3.combined.filtered.vcf.gz.tbi
SNP_INDEL_Pf3D7_07_v3.combined.filtered.vcf.gz
SNP_INDEL_Pf3D7_07_v3.combined.filtered.vcf.gz.tbi
SNP_INDEL_Pf3D7_08_v3.combined.filtered.vcf.gz
SNP_INDEL_Pf3D7_08_v3.combined.filtered.vcf.gz.tbi
SNP_INDEL_Pf3D7_09_v3.combined.filtered.vcf.gz
SNP_INDEL_Pf3D7_09_v3.combined.filtered.vcf.gz.tbi
SNP_INDEL_Pf3D7_10_v3.combined.filtered.vcf.gz
SNP_

In [48]:
# sanity check
combined_filtered_fn = "%s/SNP_INDEL_%s.combined.filtered.vcf.gz" % (release5_final_files_dir, 'Pf_M76611')
!zgrep -v '^#' {combined_filtered_fn} | wc -l
!zgrep -v '^#' {combined_filtered_fn} | grep PASS | wc -l
combined_filtered_fn = "%s/SNP_INDEL_%s.combined.filtered.vcf.gz" % (release5_final_files_dir, 'Pf3D7_API_v3')
!zgrep -v '^#' {combined_filtered_fn} | wc -l
!zgrep -v '^#' {combined_filtered_fn} | grep PASS | wc -l


1041
0
3935
2303


In [51]:
# Sanity check on chr1. 20160205_release5_expected_variants.ipynb gives estimate of 22837+19672 = 42509 PASS variants
# but this is an underestimate due to is_snp bug in vcfnp
combined_filtered_fn = "%s/SNP_INDEL_%s.combined.filtered.vcf.gz" % (release5_final_files_dir, 'Pf3D7_01_v3')
!zgrep -v '^#' {combined_filtered_fn} | wc -l
!zgrep -v '^#' {combined_filtered_fn} | grep PASS | wc -l


159946
45117


In [53]:
# Sanity check on chr5. 20160205_release5_expected_variants.ipynb gives estimate of 65425+43780 = 109205 PASS variants
# but this is an underestimate due to is_snp bug in vcfnp
combined_filtered_fn = "%s/SNP_INDEL_%s.combined.filtered.vcf.gz" % (release5_final_files_dir, 'Pf3D7_05_v3')
!zgrep -v '^#' {combined_filtered_fn} | wc -l
!zgrep -v '^#' {combined_filtered_fn} | grep PASS | wc -l


196674
115681


In [58]:
# Sanity check on chr14. 20160205_release5_expected_variants.ipynb gives estimate of 197478+99568 = 297046 PASS variants
# but this is an underestimate due to is_snp bug in vcfnp
combined_filtered_fn = "%s/SNP_INDEL_%s.combined.filtered.vcf.gz" % (release5_final_files_dir, 'Pf3D7_14_v3')
!zgrep -v '^#' {combined_filtered_fn} | wc -l
!zgrep -v '^#' {combined_filtered_fn} | grep PASS | wc -l


500801
313307
